In [74]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dset

import matplotlib.pyplot as plt
import pickle
import os

In [70]:
class MultipleSourceDataSet(Dataset):
    
    def __init__(self, json_file, root_dir, transform = None):
        
        with open(root_dir + 'block0.json') as f:
            self.result = torch.Tensor(json.load(f))
            
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.result[0])
    
    def __getitem__(self):
        None

In [71]:
import json
root_dir = '/home/tk/Documents/blocks/'
with open (root_dir + 'block0.json') as f:
    result = json.load(f)
    
with open (root_dir + 'block1.json') as f:
    result1 = json.load(f)

In [78]:
len(x)

10

In [17]:
trans = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (1.0,))
    ])

trainset = dset.MNIST(root = '../data', train = True, transform = trans, download = True)
testset = dset.MNIST(root = '../data', train = False, transform = trans)

In [18]:
trainloader = torch.utils.data.DataLoader(dataset = trainset,
                                                batch_size = 8,
                                                shuffle = True)

testloader = torch.utils.data.DataLoader(dataset = testset,
                                               batch_size = 8,
                                               shuffle = True)

In [24]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 500)
        self.fc2 = nn.Linear(500, 256)
        self.fc3 = nn.Linear(256, 10)

        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
model = Net()
print (model)

Net(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)


In [36]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.008, momentum = 0.03)

In [43]:
model.train()
for epoch in range(10):
    running_loss = 0
    
    for i, data in enumerate(trainloader, 0):
        
        inputs, labels = data
#         print (data[0].shape)
#         print (data[1].shape)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
#         if i % 2000 == 0:
#             print (loss.item())
        running_loss += loss.item()
        if i % 1000 == 0:
            print ('[%d, %5d] loss: %.3f' % (epoch, i, running_loss/ 1000))
            running_loss = 0
            
torch.save(model, 'FeatureNet.pkl')

[0,     0] loss: 0.000
[0,  1000] loss: 0.040
[0,  2000] loss: 0.039
[0,  3000] loss: 0.043
[0,  4000] loss: 0.043
[0,  5000] loss: 0.039
[0,  6000] loss: 0.048
[0,  7000] loss: 0.043
[1,     0] loss: 0.000
[1,  1000] loss: 0.033
[1,  2000] loss: 0.038
[1,  3000] loss: 0.034
[1,  4000] loss: 0.035
[1,  5000] loss: 0.042
[1,  6000] loss: 0.038
[1,  7000] loss: 0.040
[2,     0] loss: 0.000
[2,  1000] loss: 0.036
[2,  2000] loss: 0.033
[2,  3000] loss: 0.031
[2,  4000] loss: 0.034
[2,  5000] loss: 0.036
[2,  6000] loss: 0.031
[2,  7000] loss: 0.037
[3,     0] loss: 0.000
[3,  1000] loss: 0.028
[3,  2000] loss: 0.032
[3,  3000] loss: 0.031
[3,  4000] loss: 0.024
[3,  5000] loss: 0.028
[3,  6000] loss: 0.031
[3,  7000] loss: 0.030
[4,     0] loss: 0.000
[4,  1000] loss: 0.023
[4,  2000] loss: 0.025
[4,  3000] loss: 0.030
[4,  4000] loss: 0.027
[4,  5000] loss: 0.029
[4,  6000] loss: 0.029
[4,  7000] loss: 0.026
[5,     0] loss: 0.000
[5,  1000] loss: 0.025
[5,  2000] loss: 0.025
[5,  3000] 

/home/tk/anaconda3/lib/python3.7/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [40]:
li = ['a', 'b', 'c', 'd']

In [42]:
for i , value in enumerate(li, 2):
    print (i, value)

2 a
3 b
4 c
5 d


In [62]:
def train():
    model.train()
    
    for i, data in enumerate(data_loader_train, 0):
        inputs, labels = data
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [63]:
def val():
    model.eval()
    
    for i, data in enumerate(data_loader_test, 0):
        inputs, labels = data
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)

In [69]:
for epo in range(epochs):
    
    running_loss = 0
    for i, data in enumerate(trainloader, 0):
        
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 5 == 4:
            print ('loss:', running_loss)
        


ValueError: Expected input batch_size (16) to match target batch_size (64).

In [72]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [70]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')